## Imports:

In [1]:
# imports
import csv

from collections import defaultdict

import pandas as pd
pd.set_option('display.max_columns', 60)

import munch

from IPython.display import display, HTML

import pybedtools as pbt

import sh

import tabulate
tbl = tabulate.tabulate

from spartan.utils.annotations.ensembl.gff3 import parse_gff3
from spartan.utils.annotations.ensembl.gff3 import parse_gff3_attributes

from spartan.utils.fastas import ParseFastA

from spartan.utils.files import tableFile2namedTuple

from spartan.utils.genome_specific.GfusI1 import GfusI1_0

from gs_ddRAD2015.scripts import ld_figures as ldfigs

## File paths:

In [2]:
# File Paths

## LD analysis dir
ld_dir = "/home/gus/data/gs_2015_ld/OT_MS_NB_indiv.geno.ld"


## Fasta file for renaming contigs ----------------------------------------------------------
fasta = "/home/gus/remote_mounts/louise/data/genomes/glossina_fuscipes/assemblies/GfusI1/Glossina-fuscipes-IAEA_SCAFFOLDS_GfusI1.fa"

## Functional annatation (Argot2)
fanno = "/home/gus/remote_mounts/louise/data/genomes/glossina_fuscipes/annotations/functional/GfusI1.1_pre/argot2_out/argot_functional_annotations_ts150.h5"

## For setting up the BEDTOOLS phase
btools_gene_models_gff3 = "/home/gus/remote_mounts/louise/data/genomes/glossina_fuscipes/annotations/Glossina-fuscipes-IAEA_BASEFEATURES_GfusI1.1.gff3"

## Input BEDs etc if using files
Top10_InfectionOverall_path = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top10_InfectionOverall.bed"
Selected_PopPairwiseMSOT_Environm_path = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection/Selected_PopPairwiseMSOT_Environm.bed"
Selected_PopPairwiseMSNB_Environm_path = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection/Selected_PopPairwiseMSNB_Environm.bed"
Top10_corrected_PopPairwiseOverlap_Infection2015_path = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-07-23_overlap_infection/Top10_corrected_PopPairwiseOverlap_Infection2015.bed"
Top10_PopPairwiseOverlap_Environm_path= "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top10_PopPairwiseOverlap_Environm.bed"


In [3]:
# manual definition of bed data:

pcadapt_outliers_str = """Scaffold150	13772	13773
Scaffold150	13771	13772
Scaffold250	396213	396214
Scaffold144	565216	565217
Scaffold207	475991	475992
Scaffold368	310506	310507"""

MS_bayescan_infection_outlier_str = """JFJR01006593.1\t85293\t85294"""

In [4]:
# load LD analysis results
figs = ldfigs.Figures(out_dir=ld_dir, formats='png')
figs.load_data_tables()
ld_tables = munch.Munch(figs.d)
ld_tables.keys()


ld_figures: loading data_tables.




['ld',
 'd_bins',
 'contig_info',
 'd_bin_v_others_melt',
 'len_contigs_per_bin',
 'd_bin_v_others',
 'contigs_per_bin',
 'mean_bin_r2_all',
 'sp_contigs',
 'ld_contig']

In [5]:
ld = ld_tables.ld
del(ld_tables)

In [6]:
# load BED files
Top10_InfectionOverall = pbt.BedTool(Top10_InfectionOverall_path)
Selected_PopPairwiseMSOT_Environm = pbt.BedTool(Selected_PopPairwiseMSOT_Environm_path)
Selected_PopPairwiseMSNB_Environm = pbt.BedTool(Selected_PopPairwiseMSNB_Environm_path)
Top10_corrected_PopPairwiseOverlap_Infection2015 = pbt.BedTool(Top10_corrected_PopPairwiseOverlap_Infection2015_path)
Top10_PopPairwiseOverlap_Environm = pbt.BedTool(Top10_PopPairwiseOverlap_Environm_path)

# load BED strings
pcadapt_outliers = pbt.BedTool(pcadapt_outliers_str,from_string=True)
MS_bayescan_infection_outlier = pbt.BedTool(MS_bayescan_infection_outlier_str,from_string=True)

In [7]:
# load gene models into pybedtools object and filter for only gene features
btools_gene_models_pbt = pbt.BedTool(btools_gene_models_gff3)
genes = btools_gene_models_pbt.filter(lambda x: x[2] == 'gene').saveas()

In [8]:
# load functional annotations
argot2 = pd.read_hdf(path_or_buf=fanno, key='dataframe')

In [9]:
argot2['gene_id'] = argot2.Sequence.apply(lambda x: x[:-3])
argot2_200 = argot2[argot2['Total Score'] >= 200]
argot2.head()

,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content,gene_id
0,GFUI034947-PA,P,GO:0006508,proteolysis,270.313447,0.496543,8.247696,GFUI034947
1,GFUI035874-PA,F,GO:0005515,protein binding,529.038456,0.500000,5.471582,GFUI035874
2,GFUI033625-PA,P,GO:0005980,glycogen catabolic process,307.758251,0.284597,13.424450,GFUI033625
3,GFUI033625-PA,F,GO:0004134,4-alpha-glucanotransferase activity,159.513252,0.177063,10.153643,GFUI033625
4,GFUI033625-PA,F,GO:0004135,"amylo-alpha-1,6-glucosidase activity",184.352303,0.177063,11.734746,GFUI033625


#### Function to create dictionary-based retrieval object for gene/SNP data:

In [10]:
def snp_vs_gff_to_DF(bedtools_out):
    headers = ["bed3_seq",
               "bed3_start",
               "bed3_end",
               "gff3_seq",
               "gff3_source",
               "gff3_type",
               "gff3_start",
               "gff3_end",
               "gff3_score",
               "gff3_strand",
               "gff3_phase",
               "gff3_attributes",]
    df = pd.read_csv(bedtools_out.fn, sep='\t', names=headers)
    
    gene_id = lambda x: parse_gff3_attributes(x)['ID']
    
    df['gff3_rec'] = df.gff3_attributes.apply(gene_id)
    
    return df

def genes_near_bed(query_bedtool, gene_bedtool, annotations, w=1000):
    df = snp_vs_gff_to_DF(query_bedtool.window(gene_bedtool, w=w))
    df = df.merge(right=annotations,
                  how='left',
                  on=None,
                  left_on='gff3_rec', 
                  right_on='gene_id')
    return df

In [11]:
# - generate set of non-redundant andrea snps from joined table as set of tuples
# - convert to bedtool object(s)
def reduce_joined_SNP_coords(snps_interest_df):
    df = snps_interest_df  # less typing

    if len(df) == 0:
        return None
    
    
    snp_set = set()
    snp_set.update(list(df.apply(lambda x: str((x.seq_x, x.start_x, x.end_x)),1)))
    snp_set.update(list(df.apply(lambda x: str((x.seq_y, x.start_y, x.end_y)),1)))

    snp_set.discard("(nan, nan, nan)")
    
    return pbt.BedTool('\n'.join((l.replace(".0",'').replace("'","").replace('(','').replace(')','').replace(',','\t') for l in snp_set)), from_string=True)


# - get windowed intersection intersection with gene models using pybedtools

def get_win_isec(a,b,win=1000):
    if a is None:
        return None
    return a.window(b, w=win)


def add_SNP_to_gene_distance(df):
    if df is not None:
        df['d_to_gene'] = df.apply(d_to_gene,1)


def d_to_gene(x):

    low,high = ((x.bed3_end - x.gff3_start),(x.bed3_end - x.gff3_end))

    if (low >= 0) and (high <= 0):
        return 0
    else:
        return min([abs(low),abs(high)])

    

# get annotations for genes:
def join_genes_with_annos(genes, annotations, how="inner"):
    if genes is None:
        return None
    return pd.merge(left=genes, right=annotations, 
                    how=how, 
                    on=None, 
                    left_on='gff3_rec', right_on='gene_id')

In [12]:
def make_table_selected_snps_vs_LD_filter(anno_dfs):
    
    new_headers = ["SNP group",
                   "Gene",
                   "Scaffold",
                   "SNP Location",
                   "Distance to Gene",
                   "Top C",
                   "Top P",
                   "Top F"]

    old_headers = ["snp_group",
                   "bed3_seq",
                   "gff3_rec",
                   "bed3_end",
                   "d_to_gene"]


    
    concat_df = concat_dfs(anno_dfs)


    # group concat_df
    groups = concat_df.groupby(by=old_headers)

    # Collect row data
    out_rows = []

    for name,group in groups.groups.items():
        out_rows.append(new_row_from_group(groups.get_group(name)))

    # create final table
#     out_table = pd.DataFrame(data=out_rows, index=None, columns=new_headers, dtype=None, copy=False)
#     gene_mask = out_table.Gene.notnull()
#     out_table = out_table[gene_mask]
    return pd.DataFrame(data=out_rows, index=None, columns=new_headers, dtype=None, copy=False)



def new_row_from_group(group):

    row  = get_old_header_values(group)
    row["Top C"] = get_top_3_terms(group=group,aspect="C")
    row["Top P"] = get_top_3_terms(group=group,aspect="P")
    row["Top F"] = get_top_3_terms(group=group,aspect="F")

    return pd.Series(row)

def get_old_header_values(group):
    vals = munch.Munch()

    vals["SNP group"] = group.snp_group.iloc[0]
    vals["Scaffold"] = group.bed3_seq.iloc[0]
    vals["Gene"] = group.gff3_rec.iloc[0]
    vals["SNP Location"] = group.bed3_end.iloc[0]
    vals["Distance to Gene"] = group.d_to_gene.iloc[0]

    return vals


def get_top_3_terms(group,aspect):
    df = group.query("Aspect == '{aspect}'".format(aspect=aspect))

    return ";".join(df.sort("Total Score").Name[:3].values)

def concat_dfs(dfs):

    concat_df = pd.DataFrame()

    # add snp-group name to rows and concat to new table
    for name, df in dfs.items():

        try:
            df["snp_group"] = name
        except TypeError:
            pass

        concat_df = pd.concat(objs=[concat_df,df], axis=0, join='outer', join_axes=None, ignore_index=True, keys=None, levels=None, names=None, verify_integrity=False, copy=True)

    return concat_df

#  Genes located near by `query_bedtool`

In [13]:
NEARBY_GENES = {}
NEARBY_GENES['Top10_InfectionOverall'] = genes_near_bed(query_bedtool=Top10_InfectionOverall, gene_bedtool=genes, annotations=argot2_200)
NEARBY_GENES['Top10_corrected_PopPairwiseOverlap_Infection2015'] = genes_near_bed(query_bedtool=Top10_corrected_PopPairwiseOverlap_Infection2015, gene_bedtool=genes, annotations=argot2_200)
NEARBY_GENES['Top10_PopPairwiseOverlap_Environm'] = genes_near_bed(query_bedtool=Top10_PopPairwiseOverlap_Environm, gene_bedtool=genes, annotations=argot2_200)
NEARBY_GENES['Selected_PopPairwiseMSOT_Environm'] = genes_near_bed(query_bedtool=Selected_PopPairwiseMSOT_Environm, gene_bedtool=genes, annotations=argot2_200)
NEARBY_GENES['Selected_PopPairwiseMSNB_Environm'] = genes_near_bed(query_bedtool=Selected_PopPairwiseMSNB_Environm, gene_bedtool=genes, annotations=argot2_200)
NEARBY_GENES['pcadapt_outliers'] = genes_near_bed(query_bedtool=pcadapt_outliers, gene_bedtool=genes, annotations=argot2_200)
NEARBY_GENES['MS_bayescan_infection_outlier'] = genes_near_bed(query_bedtool=MS_bayescan_infection_outlier, gene_bedtool=genes, annotations=argot2_200)


In [14]:
NEARBY_GENES['MS_bayescan_infection_outlier'].head()

,bed3_seq,bed3_start,bed3_end,gff3_seq,gff3_source,gff3_type,gff3_start,gff3_end,gff3_score,gff3_strand,gff3_phase,gff3_attributes,gff3_rec,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content,gene_id


In [15]:
for name, value in NEARBY_GENES.items():
        if value is None:
            pass
        elif (len(value) > 0):
            add_SNP_to_gene_distance(value)
        else:
            NEARBY_GENES[name] = None

In [16]:
table1 = make_table_selected_snps_vs_LD_filter(NEARBY_GENES)

In [1]:
table1[table1["SNP group"] == "Top10_PopPairwiseOverlap_Environm"].head(15)

NameError: name 'table1' is not defined

In [18]:
# convert LD filtered SNP-pairs into table of single seq:location data

ld_filtered_snp_pairs = ld.query("one_minus_cdf_BH <= 0.01")
ld_filtered_snps_a = ld_filtered_snp_pairs[["CHR_A","BP_A"]].rename(columns={"CHR_A":"Scaffold","BP_A":"SNP Location"})
ld_filtered_snps_b = ld_filtered_snp_pairs[["CHR_A","BP_B"]].rename(columns={"CHR_A":"Scaffold","BP_B":"SNP Location"})
ld_filtered_snps = pd.concat([ld_filtered_snps_a,ld_filtered_snps_b]).sort()
ld_filtered_snps["LD Filtered"] = 'True'
ld_filtered_snps.head()

,Scaffold,SNP Location,LD Filtered
11,Scaffold0,13388,True
11,Scaffold0,86267,True
21,Scaffold0,183680,True
21,Scaffold0,13388,True
84,Scaffold0,13388,True


In [19]:
# What is the passed/not passed data for LD_filter?

total_pairs = len(ld)
filtered_pairs = len(ld_filtered_snp_pairs)

print "Total SNP-pairs: {}".format(total_pairs)
print "Filtered SNP-pairs: {}".format(filtered_pairs)
print "Percent filtered_pairs to total_pairs: {}".format(float(filtered_pairs)/total_pairs*100)


Total SNP-pairs: 6454294
Filtered SNP-pairs: 24372
Percent filtered_pairs to total_pairs: 0.377609076996


In [20]:
table1 = table1.merge(ld_filtered_snps, 
             how='left', 
             on=["Scaffold","SNP Location"], left_on=None, right_on=None,  
             sort=False, suffixes=('_x', '_y'), copy=True).drop_duplicates()

In [21]:
# table1.to_excel('/home/gus/src/repos/git/markdown-docs/manuscripts/Gloria-Soria/2015/ddRAD58/tables/report_to_andrea_to_authors_PREFORMAT.xls',index=False)

In [22]:
# table1.query()

# Confirm SNP numbers for `Selected_PopPairwiseMSOT_Environm` and `Selected_PopPairwiseMSNB_Environm`

In [23]:
# number of SNPs total
Selected_PopPairwiseMSOT_Environm_count = len(Selected_PopPairwiseMSOT_Environm)
Selected_PopPairwiseMSNB_Environm_count = len(Selected_PopPairwiseMSNB_Environm)

In [24]:
# number of SNPs passed LD
Selected_PopPairwiseMSOT_Environm_count_LD = Selected_PopPairwiseMSOT_Environm.to_dataframe().merge(ld_filtered_snps, 
             how='inner', 
             on=None, left_on=["chrom","end"], right_on=["Scaffold","SNP Location"],  
             sort=False, suffixes=('_x', '_y'), copy=True).drop_duplicates()
Selected_PopPairwiseMSNB_Environm_count_LD = Selected_PopPairwiseMSNB_Environm.to_dataframe().merge(ld_filtered_snps, 
             how='inner', 
             on=None, left_on=["chrom","end"], right_on=["Scaffold","SNP Location"],  
             sort=False, suffixes=('_x', '_y'), copy=True).drop_duplicates()

In [25]:
msg = """Selected_PopPairwiseMSOT_Environm_count_LD: {Selected_PopPairwiseMSOT_Environm_count_LD}
Selected_PopPairwiseMSNB_Environm_count_LD: {Selected_PopPairwiseMSNB_Environm_count_LD}"""

print msg.format(Selected_PopPairwiseMSOT_Environm_count_LD=len(Selected_PopPairwiseMSOT_Environm_count_LD),
Selected_PopPairwiseMSNB_Environm_count_LD= len(Selected_PopPairwiseMSNB_Environm_count_LD))

Selected_PopPairwiseMSOT_Environm_count_LD: 60
Selected_PopPairwiseMSNB_Environm_count_LD: 8


In [26]:
Selected_PopPairwiseMSOT_Environm_count_LD.head()

,chrom,start,end,Scaffold,SNP Location,LD Filtered
0,Scaffold2,1124644,1124645,Scaffold2,1124645,True
1,Scaffold2,1134591,1134592,Scaffold2,1134592,True
5,Scaffold87,761672,761673,Scaffold87,761673,True
7,Scaffold319,155546,155547,Scaffold319,155547,True
11,Scaffold319,195699,195700,Scaffold319,195700,True


In [27]:
# number of SNPs passed LD
Selected_PopPairwiseMSOT_Environm_count_LD
Selected_PopPairwiseMSNB_Environm_count_LD

,chrom,start,end,Scaffold,SNP Location,LD Filtered
0,Scaffold288,11420,11421,Scaffold288,11421,True
1,Scaffold549,36666,36667,Scaffold549,36667,True
2,Scaffold549,36672,36673,Scaffold549,36673,True
3,Scaffold368,277080,277081,Scaffold368,277081,True
5,Scaffold368,279445,279446,Scaffold368,279446,True
6,Scaffold368,279478,279479,Scaffold368,279479,True
9,Scaffold368,310506,310507,Scaffold368,310507,True
10,Scaffold368,310519,310520,Scaffold368,310520,True


In [28]:
Selected_PopPairwiseMSOT_Environm.to_dataframe().head()

,chrom,start,end
0,Scaffold2,1124644,1124645
1,Scaffold2,1134591,1134592
2,Scaffold17,839513,839514
3,Scaffold41,1051694,1051695
4,Scaffold77,83136,83137


# Overlap between top 10 infect and geographical

In [29]:
env10 = NEARBY_GENES['Top10_PopPairwiseOverlap_Environm']
inf10 = NEARBY_GENES['Top10_corrected_PopPairwiseOverlap_Infection2015']
inf10.head()

,bed3_seq,bed3_start,bed3_end,gff3_seq,gff3_source,gff3_type,gff3_start,gff3_end,gff3_score,gff3_strand,gff3_phase,gff3_attributes,gff3_rec,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content,gene_id,d_to_gene,snp_group
0,Scaffold2,2593208,2593209,Scaffold2,VectorBase,gene,2593175,2594101,.,-,.,ID=GFUI023810;biotype=protein_coding,GFUI023810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Top10_corrected_PopPairwiseOverlap_Infection2015
1,Scaffold355,306767,306768,Scaffold355,VectorBase,gene,287768,311543,.,-,.,ID=GFUI027633;biotype=protein_coding,GFUI027633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Top10_corrected_PopPairwiseOverlap_Infection2015
2,Scaffold163,513433,513434,Scaffold163,VectorBase,gene,513883,527197,.,-,.,ID=GFUI010652;biotype=protein_coding,GFUI010652,GFUI010652-PA,P,GO:0006200,ATP catabolic process,474.917555,0.939484,8.421864,GFUI010652,449,Top10_corrected_PopPairwiseOverlap_Infection2015
3,Scaffold163,513433,513434,Scaffold163,VectorBase,gene,513883,527197,.,-,.,ID=GFUI010652;biotype=protein_coding,GFUI010652,GFUI010652-PA,C,GO:0019028,viral capsid,1239.312392,1.000000,6.609099,GFUI010652,449,Top10_corrected_PopPairwiseOverlap_Infection2015
4,Scaffold873,18833,18834,Scaffold873,VectorBase,gene,11605,18148,.,+,.,ID=GFUI050068;biotype=protein_coding,GFUI050068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,686,Top10_corrected_PopPairwiseOverlap_Infection2015


In [30]:
env_inf = env10.merge(right=inf10, 
                    how='inner',
                    on=["bed3_seq","bed3_start","bed3_end"], left_on=None, right_on=None,
                    left_index=False, right_index=False,
                    sort=False, suffixes=('_x', '_y'),
                    copy=True)

In [31]:
env_inf

,bed3_seq,bed3_start,bed3_end,gff3_seq_x,gff3_source_x,gff3_type_x,gff3_start_x,gff3_end_x,gff3_score_x,gff3_strand_x,gff3_phase_x,gff3_attributes_x,gff3_rec_x,Sequence_x,Aspect_x,GO ID_x,Name_x,Total Score_x,Internal Confidence_x,Information Content_x,gene_id_x,d_to_gene_x,snp_group_x,gff3_seq_y,gff3_source_y,gff3_type_y,gff3_start_y,gff3_end_y,gff3_score_y,gff3_strand_y,gff3_phase_y,gff3_attributes_y,gff3_rec_y,Sequence_y,Aspect_y,GO ID_y,Name_y,Total Score_y,Internal Confidence_y,Information Content_y,gene_id_y,d_to_gene_y,snp_group_y


In [32]:
env10_set = set(env10.bed3_end)
inf10_set = set(inf10.bed3_end)

In [33]:
env10_set.intersection(inf10_set)

set()

In [34]:
inf10.query("gff3_rec == 'GFUI009292'")

,bed3_seq,bed3_start,bed3_end,gff3_seq,gff3_source,gff3_type,gff3_start,gff3_end,gff3_score,gff3_strand,gff3_phase,gff3_attributes,gff3_rec,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content,gene_id,d_to_gene,snp_group


# Do ANY of the PCAdapt SNPs have LD filtered pairs?

In [35]:
pcadapt_outliers_df = pcadapt_outliers.to_dataframe()
pcadapt_outliers_df.head()

,chrom,start,end
0,Scaffold150,13772,13773
1,Scaffold150,13771,13772
2,Scaffold250,396213,396214
3,Scaffold144,565216,565217
4,Scaffold207,475991,475992


In [36]:
pcadapt_outliers_df.merge(right=ld_filtered_snps, 
                          how='left', 
                          on=None, left_on=["chrom","end"], right_on=["Scaffold","SNP Location"], 
                          left_index=False, right_index=False, 
                          sort=False, 
                          suffixes=('_x', '_y'), 
                          copy=True)

,chrom,start,end,Scaffold,SNP Location,LD Filtered
0,Scaffold150,13772,13773,NaN,NaN,NaN
1,Scaffold150,13771,13772,NaN,NaN,NaN
2,Scaffold250,396213,396214,NaN,NaN,NaN
3,Scaffold144,565216,565217,NaN,NaN,NaN
4,Scaffold207,475991,475992,NaN,NaN,NaN
5,Scaffold368,310506,310507,Scaffold368,310507,True


In [37]:
pcadapt_ld_pair = ld_filtered_snp_pairs.query("CHR_A == 'Scaffold368'").query("(BP_A == 310507) or (BP_B == 310507)")
pcadapt_ld_pair

,Unnamed: 0,Unnamed: 0.1,CHR_A,BP_A,BP_B,N_INDV,R2,BP_DELTA,distance_bin,R2_scaled_for_B,alpha_param,beta_param,cdf,one_minus_cdf,one_minus_cdf_BH,MAP_succeeded
5365233,5365233,5365233,Scaffold368,279479,310507,47,1,31028,31000,0.9995,NaN,NaN,1,1.708366e-08,0.000002,False


In [38]:
pcadapt_ld_pair.to_clipboard()

## Any interesting genes near the paired SNP?

In [39]:
pcadapt_ld_mate = pbt.BedTool("Scaffold368\t279478\t279479", from_string=True)

In [40]:
genes_near_bed(query_bedtool=pcadapt_ld_mate, gene_bedtool=genes, annotations=argot2_200)

,bed3_seq,bed3_start,bed3_end,gff3_seq,gff3_source,gff3_type,gff3_start,gff3_end,gff3_score,gff3_strand,gff3_phase,gff3_attributes,gff3_rec,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content,gene_id


----------------

# MS_bayescan_infection_outlier

### Any interesting genes near the main SNP?

In [41]:
NEARBY_GENES['MS_bayescan_infection_outlier'].head()

AttributeError: 'NoneType' object has no attribute 'head'

Answer seems to be "no" becuase `NEARBY_GENES['MS_bayescan_infection_outlier'] is None`.

# Is `MS_bayescan_infection_outlier` in an LD filtered pair?

In [42]:
MS_bayescan_infection_outlier_df = MS_bayescan_infection_outlier.to_dataframe()
MS_bayescan_infection_outlier_df.head()

,chrom,start,end
0,JFJR01006593.1,85293,85294


In [43]:
MS_bayescan_infection_outlier_df.merge(right=ld_filtered_snps, 
                          how='left', 
                          on=None, left_on=["chrom","end"], right_on=["Scaffold","SNP Location"], 
                          left_index=False, right_index=False, 
                          sort=False, 
                          suffixes=('_x', '_y'), 
                          copy=True)

,chrom,start,end,Scaffold,SNP Location,LD Filtered
0,JFJR01006593.1,85293,85294,NaN,NaN,NaN
